In [1]:
import math
import numpy as np
import pandas as pd
import random
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification

In [2]:
df = pd.read_excel("/workspace/output.xlsx")
df = df.iloc[:,1:]
df.fillna('', inplace=True)
df.head()

Score                                      Market.01a_OE  \
0      3  Number of people inside the shop, number of pe...   
1      0          what causes a door to open automatically.   
2      1  How many people are going through the doors. H...   
3      1  The groups of customers that might go in, the ...   
4      1  How big is the store, as a team of constructio...   

                                       Market.01b_OE  \
0  Number of people inside the shop and number of...   
1        what can cause a door to open automatically   
2  How long it will take for everyone to go throu...   
3                     customers that go in and leave   
4  What does the owner want. How tall the store i...   

                                       Market.01c_OE  
0  We will definitively need to know the amount o...  
1  i think this piece of information is important...  
2  We must know this information to decrease the ...  
3  Because we need to how many customers go in, a...  
4  This can impact how the finished product will ...

In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, DistilBertPreTrainedModel, RobertaForSequenceClassification
#from transformers.modeling_distilbert import DistilBertModel, DistilBertPreTrainedModel
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


class MultimodalRoberta(torch.nn.Module):
    def __init__(self, num_labels=5):
        super(MultimodalRoberta, self).__init__()
        self.num_labels = num_labels
        self.roberta1 = RobertaModel.from_pretrained('distilroberta-base')
        self.roberta2 = RobertaModel.from_pretrained('distilroberta-base')
        self.roberta3 = RobertaModel.from_pretrained('distilroberta-base')
        self.classifier = nn.Linear(self.roberta1.config.hidden_size + self.roberta2.config.hidden_size + self.roberta3.config.hidden_size, num_labels)

    def forward(self, input_ids_a, attention_mask_a, input_ids_b, attention_mask_b, input_ids_c, attention_mask_c):
        output_a = self.roberta1(input_ids=input_ids_a, attention_mask=attention_mask_a)
        output_b = self.roberta2(input_ids=input_ids_b, attention_mask=attention_mask_b)
        output_c = self.roberta3(input_ids=input_ids_c, attention_mask=attention_mask_c)

        concatenated_output = torch.cat((output_a.pooler_output, output_b.pooler_output, output_c.pooler_output), 1)

        return self.classifier(concatenated_output)


class MultimodalRobertaDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.columns = ["Market.01a_OE", "Market.01b_OE", "Market.01c_OE"]
        self.labels = self.dataframe['Score'].values

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        # Selecting sentence1 and sentence2 at the specified index in the data frame
        row = self.dataframe.iloc[index]
        response_a = row['Market.01a_OE']
        response_b = row['Market.01b_OE']
        response_c = row['Market.01c_OE']
        score = row['Score']

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoding_a = self.tokenizer.encode_plus(response_a, add_special_tokens=True, max_length=self.max_length, padding='max_length', return_attention_mask=True, truncation=True)
        encoding_b = self.tokenizer.encode_plus(response_b, add_special_tokens=True, max_length=self.max_length, padding='max_length', return_attention_mask=True, truncation=True)
        encoding_c = self.tokenizer.encode_plus(response_c, add_special_tokens=True, max_length=self.max_length, padding='max_length', return_attention_mask=True, truncation=True)

        return {
            'input_ids_a': torch.tensor(encoding_a['input_ids'], dtype=torch.long),
            'attention_mask_a': torch.tensor(encoding_a['attention_mask'], dtype=torch.long),
            'input_ids_b': torch.tensor(encoding_b['input_ids'], dtype=torch.long),
            'attention_mask_b': torch.tensor(encoding_b['attention_mask'], dtype=torch.long),
            'input_ids_c': torch.tensor(encoding_c['input_ids'], dtype=torch.long),
            'attention_mask_c': torch.tensor(encoding_c['attention_mask'], dtype=torch.long),
            'score': torch.tensor(score, dtype=torch.float)
        }

In [4]:
# Use GPU
from torch.utils.data import Dataset, DataLoader

tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base', output_attentions=False)
train_df, val_df = train_test_split(df, test_size=0.2, random_state = 42)

# Create datasets
train_dataset = MultimodalRobertaDataset(train_df, tokenizer)
val_dataset = MultimodalRobertaDataset(val_df, tokenizer)
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8)
val_loader = DataLoader(val_dataset, batch_size=8)

model = MultimodalRoberta()
device = torch.device('cuda')
model.to(device)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model traine

MultimodalRoberta(
  (roberta1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [5]:
#loss_function = nn.MSELoss()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [6]:
for epoch in range(10):
    total_loss = 0
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(
            batch['input_ids_a'].to(device), batch['attention_mask_a'].to(device),
            batch['input_ids_b'].to(device), batch['attention_mask_b'].to(device),
            batch['input_ids_c'].to(device), batch['attention_mask_c'].to(device)
        )
        loss = loss_function(outputs, batch['score'].to(device).long())
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print("Average training loss: {0:.2f}".format(avg_train_loss))

Average training loss: 1.40
Average training loss: 0.97
Average training loss: 0.67
Average training loss: 0.47
Average training loss: 0.30
Average training loss: 0.19
Average training loss: 0.11
Average training loss: 0.08
Average training loss: 0.08
Average training loss: 0.10


In [7]:
model.eval()

all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
      # Forward pass
      outputs = model(
          batch['input_ids_a'].to(device), batch['attention_mask_a'].to(device),
          batch['input_ids_b'].to(device), batch['attention_mask_b'].to(device),
          batch['input_ids_c'].to(device), batch['attention_mask_c'].to(device)
      )
      #all_predictions.extend(outputs.cpu().numpy())
      #all_labels.extend(batch['score'].numpy())

      _, outputs = torch.max(outputs, 1)
      all_predictions.extend(outputs.cpu().numpy())
      all_labels.extend(batch['score'].numpy())

# Since my predictions return float number, such as 2.3 and 3.5, I decide to round
# or map the number in the following way:
# 2.5 -> 3; 2.3 -> 2; to get a better algorithm to calculate the accuracy

def arrayround(arr,n=0):
    import numpy as np
    flag = np.where(arr>=0,1,-1)
    arr = np.abs(arr)
    arr10 = arr*10**(n+1)
    arr20 = np.floor(arr10)
    arr30 = np.where(arr20%10==5,(arr20+1)/10**(n+1),arr20/10**(n+1))
    result = np.around(arr30,n)
    return result*flag

all_predictions = np.array(all_predictions).flatten()
#all_predictions = arrayround(all_predictions)



# Compute the average accuracy over all batches
correct_predictions = sum(all_predictions == np.array(all_labels))
total_predictions = len(all_predictions)
test_accuracy = correct_predictions / total_predictions

print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

Test Accuracy: 78.20%


In [23]:
import warnings
warnings.filterwarnings("ignore")
# Define the hyperparameter search space
learning_rates = [1e-5]  # List of learning rates to try
batch_sizes = [4, 8, 16, 32]  # List of batch sizes to try
epoch_sizes = [5,6,7,8,9,10]
combination_accuracies = {}

for epoch_size in epoch_sizes:
  for batch_size in batch_sizes:
    # Re-training the model for each combination of hyperparameters
    model = MultimodalRoberta()
    device = torch.device('cuda')
    model.to(device)
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    model.train()
    print([epoch_size, batch_size]) # prints out the hyperparameter combination being tested
    for epoch in range(epoch_size):
        total_loss = 0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            outputs = model(
                batch['input_ids_a'].to(device), batch['attention_mask_a'].to(device),
                batch['input_ids_b'].to(device), batch['attention_mask_b'].to(device),
                batch['input_ids_c'].to(device), batch['attention_mask_c'].to(device)
            )
            loss = loss_function(outputs, batch['score'].to(device).long())
            total_loss += loss.item()
    
            loss.backward()
            optimizer.step()
    # Put the model in evaluation mode
    model.eval()

    # Initialize variables to keep track of predictions and ground truth labels
    all_predictions = []
    all_labels = []

    # Evaluate on the test dataset
    with torch.no_grad():
        for batch in val_loader:
          # Forward pass
          outputs = model(
              batch['input_ids_a'].to(device), batch['attention_mask_a'].to(device),
              batch['input_ids_b'].to(device), batch['attention_mask_b'].to(device),
              batch['input_ids_c'].to(device), batch['attention_mask_c'].to(device)
          )
          #all_predictions.extend(outputs.cpu().numpy())
          #all_labels.extend(batch['score'].numpy())
    
          _, outputs = torch.max(outputs, 1)
          all_predictions.extend(outputs.cpu().numpy())
          all_labels.extend(batch['score'].numpy())

    # Calculate accuracy
    all_predictions = np.array(all_predictions).flatten()

    correct_predictions = sum(all_predictions == np.array(all_labels))
    total_predictions = len(all_predictions)
    test_accuracy = correct_predictions / total_predictions
    
    print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

    combination_accuracies[(epoch_size, batch_size)] = test_accuracy

    # Deletes the cache and the model from GPU memory
    torch.cuda.empty_cache()
    del model

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_275/3967691171.py", line 12, in <module>
    model = MultimodalRoberta()
  File "/tmp/ipykernel_275/817507556.py", line 15, in __init__
    self.roberta1 = RobertaModel.from_pretrained('distilroberta-base')
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 3480, in from_pretrained
    - **summary_use_proj** (`bool`) -- Add a projection after the vector extraction.
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 3824, in _load_pretrained_model
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 571, in _load_state_dict_into_model
    state_dict_folder=None,
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 569, in load
    offload_

In [21]:
torch.cuda.empty_cache()
del model

In [13]:
val_df[['Score']]

Score
327      2
578      1
470      2
362      1
265      1
..     ...
464      0
532      2
431      1
248      1
208      0

[133 rows x 1 columns]

In [8]:
val_df['predicted_score'] = all_predictions

In [9]:
val_df

Score                                      Market.01a_OE  \
327      2  How many people are in the line, how many are ...   
578      1         How many people are in there at that time.   
470      2  how many people are currently in the store and...   
362      1  how many people are leaving the store. how man...   
265      1  The needed inputs would be, an input letting t...   
..     ...                                                ...   
464      0                                                  1   
532      2  the number of people currently inside the buil...   
431      1  The time the last person exited and the group ...   
248      1  How many people will it be able to fit in the ...   
208      0                                           the door   

                                         Market.01b_OE  \
327  When people leave count how many people are le...   
578                      How many people are in there.   
470  if there is less than 15 people in the store l...   
362  the process would be counting the people who a...   
265  The sensor would send inputs to the program th...   
..                                                 ...   
464                                                  1   
532  The number of people currently inside the buil...   
431  The estimated time people have been in the store.   
248  How many people will it be able to fit in the ...   
208                                           the door   

                                         Market.01c_OE  predicted_score  
327  once there is enough people out the store let ...                2  
578  Because if you found how many people you figur...                1  
470                         if people can go in or not                2  
362  The output would be how many people they let f...                1  
265  The outputs sent from the computer would be a ...                1  
..                                                 ...              ...  
464                                                  1                0  
532  It is most important because according to the ...                2  
431  The output would be a display on when the next...                1  
248  It's important because we need to know how man...                1  
208                                          bc they r                0  

[133 rows x 5 columns]

In [13]:
output_df.to_excel("/workspace/difference.xlsx")

In [11]:
output_df = val_df[val_df["Score"] != val_df["predicted_score"]]
output_df

Score                                      Market.01a_OE  \
443      2                        How many people are inside.   
356      1  The input would be the number of people going ...   
333      3  How many people are in the upcoming group, How...   
463      3                            amount of people inside   
72       3  Our team will need to know the amount of peopl...   
649      3  They might need to know how many people are in...   
570      1  number of people in each group, how many peopl...   
425      3  The maximum capacity of the store (15 customer...   
326      4  The inputs in this specific situation would be...   
548      3  You need to know how many people are currently...   
292      3  Number of people inside and the size of the fi...   
78       3  number of people in the store, size of next gr...   
420      2                     Inputs Group outside the door.   
503      3  the maximum number of people allowed inside th...   
363      1  Groups inside, number per group, number per gr...   
655      4  Amount of people in store and amount of people...   
658      2  How many people is the max,how to be able to t...   
176      3  How many people are currently inside the store...   
328      2                                   Number of people   
155      2  We need to know how many people are currently ...   
163      1      The sensor data, number of people in a group.   
278      3  The input would be how man people are inside t...   
615      1  When the Market is at the capacity limit, clos...   
290      3  number of people in store, number of people in...   
41       4  Number of people inside the store. Number of p...   
108      2  When a group leaves and how big that group is,...   
204      3  The number of shoppers in the store, the size ...   
299      1                           number of people outside   
452      0                                  to know the width   

                                         Market.01b_OE  \
443                            dertermening the amount   
356         The amount of people that are in the store   
333  Check how many people are currently waiting to...   
463  subtract the amount of people inside from 15, ...   
72   The most important pieces of information that ...   
649  The most important would be how many people ar...   
570                     number of people in each group   
425  Information about the number of customers curr...   
326  The process in this specific situation would b...   
548                   How many people are in the store   
292  If the number of people inside &lt; 15 &amp;&a...   
78   number of people in the store, size of next gr...   
420          it will check if the inside is too large.   
503  the program will count the amount of people in...   
363  The people leave and the door lets more people in   
655  Amount of people in store and amount of people...   
658  How to be able to tell how many people are coming   
176  The most important piece of information is how...   
328  It will process whether the number of people h...   
155  The capacity is the most important because it ...   
163                 Information coming from the sensor   
278  The door will calculate how many people can be...   
615  The capacity limit of the market and the numbe...   
290  check number of people in next group, if group...   
41   Number of people inside the store. Number of p...   
108  The most important piece of info is knowing wh...   
204                       The store's shopper capacity   
299                                 how many to let in   
452                 Get out everything and make it big   

                                         Market.01c_OE  predicted_score  
443                   How many people can come inside.                1  
356  The same amount of people that went in the sto...                2  
333  The door opens up when less than 15 people are...                2  
463       if they

In [14]:
len(val_df)

133